In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# 데이터마트용 계정 생성
# sqlDeveloper에서 진행
# create user dm_admin IDENTIFIED BY "123qwe!@#QWE";
# grant dwrole to dm_admin;
# grant unlimited tablespace to dm_admin;


In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


In [4]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

In [3]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

### JDBC

In [8]:
JDBC_DW= {
    'url':'jdbc:oracle:thin:@decorona_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA'
    , 'props':{
        'user':'dw_de'
        ,'password':'123qwe!@#QWE'
    }
}

JDBC_DM= {
    'url':'jdbc:oracle:thin:@decorona_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA'
    , 'props':{
        'user':'admin'
        ,'password':'123qwe!@#QWE'
    }
}

## Load DataMart

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수 데이터 저장

In [5]:
popu = spark.read.jdbc(url=JDBC_DW['url'],table='loc',properties=JDBC_DW['props'])
popu.show(3)

patients = spark.read.jdbc(url=JDBC_DW['url'],table='CORONA_PATIENTS',properties=JDBC_DW['props'])
patients.show(3)

+----+----+----------+------------+
| LOC|AREA|POPULATION|FACILITY_CNT|
+----+----+----------+------------+
|서울| 605|   9736027|        5224|
|부산| 770|   3396109|        1462|
|대구| 883|   2412642|        1015|
+----+----+----------+------------+
only showing top 3 rows

+------+----+---------+-------+-----------+--------+----------+
|cp_idx| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|
+------+----+---------+-------+-----------+--------+----------+
|   315|충남|      983| 768503|       3634|   36263|2022-07-28|
|   316|광주|      573| 564428|       2443|   39153|2022-07-28|
|   317|경남|     1351|1332140|       7565|   40195|2022-08-21|
+------+----+---------+-------+-----------+--------+----------+
only showing top 3 rows



In [6]:
pop_patients = popu.join(patients, on = 'loc') \
                        .select('loc'
                               ,ceil(col('population')/col('area')).alias('POPU_DENSITY')
                               ,'qur_rate'
                               ,'std_day') \
                        .orderBy(col('std_day'))

pop_patients.show()

+----+------------+--------+----------+
| loc|POPU_DENSITY|qur_rate|   std_day|
+----+------------+--------+----------+
|광주|        2920|   39153|2022-07-28|
|강원|          93|   36318|2022-07-28|
|대구|        2733|   34103|2022-07-28|
|대전|        2722|   37839|2022-07-28|
|부산|        4411|   35083|2022-07-28|
|경기|        1366|   39145|2022-07-28|
|울산|        1072|   37239|2022-07-28|
|인천|        2831|   38908|2022-07-28|
|전남|         152|   34379|2022-07-28|
|전북|         226|   35868|2022-07-28|
|제주|         378|   39131|2022-07-28|
|충남|         265|   36263|2022-07-28|
|충북|         221|   37296|2022-07-28|
|서울|       16093|   41322|2022-07-28|
|경남|         321|   35192|2022-07-28|
|세종|         811|   40272|2022-07-28|
|경북|         141|   32018|2022-07-28|
|경남|         321|   35327|2022-07-29|
|경기|        1366|   39319|2022-07-29|
|강원|          93|   36498|2022-07-29|
+----+------------+--------+----------+
only showing top 20 rows



In [9]:
pop_patients.write.jdbc(url=JDBC_DM['url'], table='CO_POPU_DENSITY',mode='append',properties=JDBC_DM['props'])

### 2.  백신접종 완료자와 코로나 확진자 수 데이터 저장

In [10]:
vaccine = spark.read.jdbc(url=JDBC_DW['url'],table='CORONA_VACCINE',properties=JDBC_DW['props'])
vaccine.show(3)

+------+----+----+-------+----------+
|CV_IDX| LOC|V_TH|  V_CNT|   STD_DAY|
+------+----+----+-------+----------+
|   221|서울|  v2|8261820|2022-09-20|
|   222|서울|  v4|1234961|2022-09-20|
|   223|서울|  v1|8340594|2022-09-20|
+------+----+----+-------+----------+
only showing top 3 rows



In [51]:
pd_vaccine = vaccine.to_pandas_on_spark()
pd_vaccine = pd_vaccine.pivot_table(index=['LOC','STD_DAY'],columns='V_TH',values='V_CNT')
pd_vaccine
pd_vaccine = pd_vaccine.reset_index()
pd_vaccine
tmp = pd_vaccine.to_spark()

vac_rate = tmp.join(popu, on='LOC') \
                    .join(patients, on='LOC') \
                        .select('loc', patients.STD_DAY
                               ,ceil(col('v1')/col('population')*100).alias('V1TH_RATE')
                               ,ceil(col('v2')/col('population')*100).alias('V2TH_RATE')
                               ,ceil(col('v3')/col('population')*100).alias('V3TH_RATE')
                               ,ceil(col('v4')/col('population')*100).alias('V4TH_RATE')
                               ,'qur_rate')

vac_rate.show(3)

,V_TH,v1,v2,v3,v4
LOC,STD_DAY,,,,
경기,2022-09-18,11828148.0000,11712435.0000,8678460.0000,1688068.0000
충북,2022-09-18,1429660.0000,1417015.0000,1106015.0000,278087.0000
울산,2022-09-18,968616.0000,959274.0000,719551.0000,125398.0000
경북,2022-09-18,2293393.0000,2269467.0000,1724688.0000,395869.0000
충남,2022-09-20,1899596.0000,1881925.0000,1475792.0000,367768.0000
대구,2022-09-18,2019298.0000,1996365.0000,1402789.0000,260381.0000
세종,2022-09-20,298623.0000,294869.0000,209215.0000,34781.0000
인천,2022-09-18,2570971.0000,2546449.0000,1908028.0000,396104.0000
서울,2022-09-18,8340437.0000,8261615.0000,6058568.0000,1231362.0000


V_TH,LOC,STD_DAY,v1,v2,v3,v4
0,경기,2022-09-18,11828148.0000,11712435.0000,8678460.0000,1688068.0000
1,충북,2022-09-18,1429660.0000,1417015.0000,1106015.0000,278087.0000
2,울산,2022-09-18,968616.0000,959274.0000,719551.0000,125398.0000
3,경북,2022-09-18,2293393.0000,2269467.0000,1724688.0000,395869.0000
4,충남,2022-09-20,1899596.0000,1881925.0000,1475792.0000,367768.0000
5,대구,2022-09-18,2019298.0000,1996365.0000,1402789.0000,260381.0000
6,세종,2022-09-20,298623.0000,294869.0000,209215.0000,34781.0000
7,인천,2022-09-18,2570971.0000,2546449.0000,1908028.0000,396104.0000
8,서울,2022-09-18,8340437.0000,8261615.0000,6058568.0000,1231362.0000
9,경남,2022-09-18,2883579.0000,2854198.0000,2142332.0000,485197.0000


+----+----------+---------+---------+---------+---------+--------+
| loc|   STD_DAY|V1TH_RATE|V2TH_RATE|V3TH_RATE|V4TH_RATE|qur_rate|
+----+----------+---------+---------+---------+---------+--------+
|강원|2022-08-14|       87|       87|       69|       18|   40330|
|강원|2022-08-02|       87|       87|       69|       18|   37126|
|강원|2022-08-27|       87|       87|       69|       18|   43269|
+----+----------+---------+---------+---------+---------+--------+
only showing top 3 rows



In [12]:
vac_rate.write.jdbc(url=JDBC_DM['url'], table='CO_VACCINE',mode='append',properties=JDBC_DM['props'])

### 3. 다중이용시설과 코로나 확진자 수의 상관관계

In [16]:
fac = spark.read.csv('/corona_data/loc/전국다중이용시설.csv', encoding='CP949', header=True)
fac.show(5)
popu = spark.read.jdbc(url=JDBC_DW['url'],table='loc',properties=JDBC_DW['props'])
popu.show()
patients = spark.read.jdbc(url=JDBC_DW['url'],table='corona_patients',properties=JDBC_DW['props'])
patients.show(5)

+----+----+------+------------------+--------------+----------------+
|연도|광역|시군구|            시설명|사업자등록번호|          시설군|
+----+----+------+------------------+--------------+----------------+
|2020|강원|강릉시|  (주)경포솔향온천|  127-86-57613|          목욕장|
|2020|강원|강릉시|    24시황실사우나|  226-33-03247|          목욕장|
|2020|강원|강릉시|      강릉고려병원|  226-82-03662|        의료기관|
|2020|강원|강릉시|강릉고속버스터미널|  219-81-01636|여객자동차터미널|
|2020|강원|강릉시|    강릉교육문화관|  226-83-03107|          도서관|
+----+----+------+------------------+--------------+----------------+
only showing top 5 rows



+----+-----+----------+------------+
| LOC| AREA|POPULATION|FACILITY_CNT|
+----+-----+----------+------------+
|서울|  605|   9736027|        5224|
|부산|  770|   3396109|        1462|
|대구|  883|   2412642|        1015|
|인천| 1065|   3014739|        1386|
|광주|  501|   1462545|         732|
|대전|  540|   1469543|         767|
|울산| 1062|   1138419|         431|
|세종|  465|    376779|         243|
|경기|10195|  13925862|        6303|
|강원|16830|   1555876|         584|
|충북| 7407|   1633472|         617|
|충남| 8246|   2181835|         824|
|전북| 8070|   1817186|         759|
|전남|12348|   1865459|         825|
|경북|19034|   2677709|         944|
|경남|10541|   3377331|        1357|
|제주| 1850|    697476|         310|
+----+-----+----------+------------+



+------+----+---------+-------+-----------+--------+----------+
|cp_idx| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|
+------+----+---------+-------+-----------+--------+----------+
|   315|충남|      983| 768503|       3634|   36263|2022-07-28|
|   316|광주|      573| 564428|       2443|   39153|2022-07-28|
|   317|경남|     1351|1332140|       7565|   40195|2022-08-21|
|   318|전남|      655| 806148|       2739|   43984|2022-09-14|
|   319|세종|       39| 162430|        978|   43676|2022-08-12|
+------+----+---------+-------+-----------+--------+----------+
only showing top 5 rows



In [27]:
fac_cnt = fac.groupBy(fac.광역.alias('loc')).agg(count('*').alias('fac_cnt'))
fac_cnt.show()
tmp = popu.join(fac_cnt, on='LOC')
tmp.show(5)
fac_popu = tmp.select('LOC', ceil(col('FACILITY_CNT')/col('POPULATION')*100000).alias('FAC_POPU'))
fac_popu.show()
co_fac = fac_popu.join(patients, on='LOC') \
                    .select('LOC'
                            , 'FAC_POPU'
                           , 'QUR_RATE'
                           , 'STD_DAY')
co_fac.show(10)

+----+-------+
| loc|fac_cnt|
+----+-------+
|경북|    944|
|대전|    767|
|전북|    759|
|충북|    617|
|울산|    431|
|경남|   1357|
|제주|    310|
|충남|    824|
|전남|    825|
|인천|   1386|
|부산|   1462|
|대구|   1015|
|경기|   6303|
|서울|   5224|
|광주|    732|
|세종|    243|
|강원|    584|
+----+-------+



+----+----+----------+------------+-------+
| LOC|AREA|POPULATION|FACILITY_CNT|fac_cnt|
+----+----+----------+------------+-------+
|서울| 605|   9736027|        5224|   5224|
|부산| 770|   3396109|        1462|   1462|
|대구| 883|   2412642|        1015|   1015|
|인천|1065|   3014739|        1386|   1386|
|광주| 501|   1462545|         732|    732|
+----+----+----------+------------+-------+
only showing top 5 rows



+----+--------+
| LOC|FAC_POPU|
+----+--------+
|서울|      54|
|부산|      43|
|대구|      43|
|인천|      46|
|광주|      50|
|대전|      53|
|울산|      38|
|세종|      65|
|경기|      46|
|강원|      38|
|충북|      38|
|충남|      38|
|전북|      42|
|전남|      45|
|경북|      36|
|경남|      41|
|제주|      45|
+----+--------+



+----+--------+--------+----------+
| LOC|FAC_POPU|QUR_RATE|   STD_DAY|
+----+--------+--------+----------+
|강원|      38|   40330|2022-08-14|
|강원|      38|   37126|2022-08-02|
|강원|      38|   43269|2022-08-27|
|강원|      38|   44783|2022-09-05|
|강원|      38|   44410|2022-09-02|
|강원|      38|   45472|2022-09-11|
|강원|      38|   36668|2022-07-30|
|강원|      38|   45443|2022-09-09|
|강원|      38|   43207|2022-08-26|
|강원|      38|   36923|2022-08-01|
+----+--------+--------+----------+
only showing top 10 rows



In [28]:
co_fac.write.jdbc(url=JDBC_DM['url'], table='CO_FACILITY',mode='append',properties=JDBC_DM['props'])

### 4. 요일별 코로나 확진자 수를 구해보자

In [37]:
patients = spark.read.jdbc(url=JDBC_DW['url'],table='CORONA_PATIENTS',properties=JDBC_DW['props'])
patients.show(10)

+------+----+---------+-------+-----------+--------+----------+
|cp_idx| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|
+------+----+---------+-------+-----------+--------+----------+
|   315|충남|      983| 768503|       3634|   36263|2022-07-28|
|   316|광주|      573| 564428|       2443|   39153|2022-07-28|
|   317|경남|     1351|1332140|       7565|   40195|2022-08-21|
|   318|전남|      655| 806148|       2739|   43984|2022-09-14|
|   319|세종|       39| 162430|        978|   43676|2022-08-12|
|   320|서울|     4954|3929537|      18158|   41322|2022-07-28|
|   321|전남|      562| 726072|       5722|   39615|2022-08-20|
|   322|전북|      902| 683947|       5481|   38277|2022-08-09|
|   323|울산|      353| 447511|       3508|   39900|2022-08-09|
|   324|대전|      722| 562623|       4119|   38741|2022-08-02|
+------+----+---------+-------+-----------+--------+----------+
only showing top 10 rows



In [53]:
day_patients = patients.withColumn('day_of_week', dayofweek(col('STD_DAY')))
pd_day_patients = day_patients.to_pandas_on_spark()
pd_day_patients = pd_day_patients.pivot_table(index=['STD_DAY'],columns='day_of_week',values='LOC_OCC_CNT')
pd_day_patients
pd_day_patients = pd_day_patients.reset_index()


# day_patients = day_patients.select('day_of_week', 'STD_DAY').show(10)
# pd_day_patients = day_patients.to_pandas_on_spark()
# pd_day_patients

day_of_week,1,2,3,4,5,6,7
STD_DAY,,,,,,,
2022-09-03,None,None,None,None,None,None,4671.9412
2022-07-30,None,None,None,None,None,None,4800.2941
2022-09-07,None,None,None,5011.5882,None,None,None
2022-08-17,None,None,None,10602.1176,None,None,None
2022-08-26,None,None,None,None,None,5928.3529,None
2022-08-25,None,None,None,None,6647.1765,None,None
2022-08-02,None,None,6542.4118,None,None,None,None
2022-09-17,None,None,None,None,None,None,2539.8235
2022-09-06,None,None,5856.7059,None,None,None,None


day_of_week,STD_DAY,1,2,3,4,5,6,7
0,2022-09-03,None,None,None,None,None,None,4671.9412
1,2022-07-30,None,None,None,None,None,None,4800.2941
2,2022-09-07,None,None,None,5011.5882,None,None,None
3,2022-08-17,None,None,None,10602.1176,None,None,None
4,2022-08-26,None,None,None,None,None,5928.3529,None
5,2022-08-25,None,None,None,None,6647.1765,None,None
6,2022-08-02,None,None,6542.4118,None,None,None,None
7,2022-09-17,None,None,None,None,None,None,2539.8235
8,2022-09-06,None,None,5856.7059,None,None,None,None
9,2022-09-19,None,1124.5882,None,None,None,None,None


day_of_week,STD_DAY,1,2,3,4,5,6,7
0,2022-09-03,None,None,None,None,None,None,4671.9412
1,2022-07-30,None,None,None,None,None,None,4800.2941
2,2022-09-07,None,None,None,5011.5882,None,None,None
3,2022-08-17,None,None,None,10602.1176,None,None,None
4,2022-08-26,None,None,None,None,None,5928.3529,None
5,2022-08-25,None,None,None,None,6647.1765,None,None
6,2022-08-02,None,None,6542.4118,None,None,None,None
7,2022-09-17,None,None,None,None,None,None,2539.8235
8,2022-09-06,None,None,5856.7059,None,None,None,None
9,2022-09-19,None,1124.5882,None,None,None,None,None


22/09/21 22:42:30 ERROR YarnScheduler: Lost executor 2 on localhost: Executor heartbeat timed out after 738583 ms
22/09/21 22:42:30 ERROR YarnScheduler: Lost executor 1 on localhost: Executor heartbeat timed out after 743577 ms
22/09/21 23:17:24 ERROR YarnScheduler: Lost executor 4 on localhost: Executor heartbeat timed out after 413477 ms
22/09/21 23:17:24 ERROR YarnScheduler: Lost executor 3 on localhost: Executor heartbeat timed out after 419792 ms
22/09/22 00:41:20 ERROR YarnScheduler: Lost executor 5 on localhost: Executor heartbeat timed out after 1375041 ms
22/09/22 00:41:20 ERROR YarnScheduler: Lost executor 6 on localhost: Executor heartbeat timed out after 1368895 ms
22/09/22 00:41:21 ERROR YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
22/09/22 00:41:21 ERROR YarnClientSchedulerBackend: Diagnostics message: Application application_1663751444661_0007 failed 2 times due to ApplicationMas